<a href="https://colab.research.google.com/github/nana-hyun/DataScienceAnalyis_study/blob/main/SL_code_dacon_Studing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

코드 따라치기

출처 : https://dacon.io/competitions/official/235896/codeshare/5005?page=1&dtype=recent

In [1]:
import os
import numpy as np
from glob import glob
import matplotlib.pyplot as plt
import seaborn as sns
import PIL
import pandas as pd
import random
import tensorflow as tf
from tensorflow.keras.layers import Dense, Dropout, Flatten, GlobalAveragePooling2D, Conv2D, MaxPool2D, ZeroPadding2D, BatchNormalization, Input, DepthwiseConv2D, Add, LeakyReLU, ReLU
from tensorflow.keras.models import Sequential,Model
from sklearn.neural_network import MLPClassifier
from tensorflow.keras.optimizers import Adam, SGD

In [2]:
train=pd.read_csv('/content/drive/MyDrive/user_data/train.csv')
test = pd.read_csv('/content/drive/MyDrive/user_data/test.csv')
sub = pd.read_csv('/content/drive/MyDrive/user_data/sample_submission.csv')

In [3]:
train['label'][train['label']=='10-1']=10
train['label'][train['label']=='10-2']=0
train['label'].astype('int')

0       0
1      10
2       3
3       8
4       9
       ..
853     9
854     1
855     4
856    10
857     7
Name: label, Length: 858, dtype: int64

In [4]:
from tqdm.notebook import tqdm

train_file_names = train["file_name"].to_numpy()
test_file_names = test["file_name"].to_numpy()
target = train["label"].to_numpy()

def load_img(file_names, target, path):   
  images = []
  for img in tqdm(file_names):
    an_img = PIL.Image.open(path + img)
    img_array = np.array(an_img)
    images.append(img_array)
  images = np.array(images)

  targets = target.copy()

  return images, targets

In [5]:
X, y = load_img(train_file_names, target, path='/content/drive/MyDrive/user_data/train/')
X_test, _ = load_img(test_file_names, np.array([None]), path='/content/drive/MyDrive/user_data/test/')

  0%|          | 0/858 [00:00<?, ?it/s]

  0%|          | 0/215 [00:00<?, ?it/s]

In [6]:
y= y.astype('int')

In [7]:
from sklearn.model_selection import train_test_split
# y의 비율을 유지하면 split
x_train, x_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [8]:
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, Flatten, GlobalAveragePooling2D, Conv2D, MaxPool2D, ZeroPadding2D, BatchNormalization, Input, DepthwiseConv2D, Add, LeakyReLU, ReLU
from tensorflow.keras.optimizers import Adam, SGD

In [9]:
IMG_SIZE = 112
from tensorflow.keras import layers
data_augmentation = tf.keras.Sequential([
  layers.experimental.preprocessing.Resizing(IMG_SIZE, IMG_SIZE), 
  layers.experimental.preprocessing.Rescaling(1./255),
  layers.experimental.preprocessing.RandomFlip("horizontal"),
  layers.experimental.preprocessing.RandomRotation(0.2),
  layers.experimental.preprocessing.RandomZoom(0.3),
  # layers.Normalization(axis=-1, mean=None, variance=None)
])

resize_and_rescale = tf.keras.Sequential([
  layers.experimental.preprocessing.Resizing(IMG_SIZE, IMG_SIZE),
  layers.experimental.preprocessing.Rescaling(1./255),
  # layers.Normalization(axis=-1, mean=None, variance=None)
])

In [10]:
agument_size = 9

agumeted_val = resize_and_rescale(x_val)
agumeted_train = resize_and_rescale(x_train)
agumeted_train_y = y_train.copy()
agumeted_val_y = y_val.copy()

for i in range(agument_size):
  agumeted_train = np.concatenate([agumeted_train, data_augmentation(x_train)])
  #agumeted_val = np.concatenate([agumeted_val, data_augmentation(x_val)])

  agumeted_train_y = np.concatenate([agumeted_train_y, y_train])
  #agumeted_val_y = np.concatenate([agumeted_val_y, y_val])

In [11]:
def build_model():


  inputs = tf.keras.layers.Input(shape=(IMG_SIZE,IMG_SIZE,3))

  outputs = Conv2D(32,(3,3),activation=None,padding='same',kernel_initializer='he_normal')(inputs)
  outputs = BatchNormalization()(outputs)
  outputs = ReLU()(outputs)
  outputs = MaxPool2D((2,2))(outputs)

  outputs = Conv2D(64,(3,3),activation=None,padding='same',kernel_initializer='he_normal')(outputs)
  outputs = BatchNormalization()(outputs)
  outputs = ReLU()(outputs)
  outputs = MaxPool2D((2,2))(outputs)

  outputs = Conv2D(128,(3,3),activation=None,padding='same',kernel_initializer='he_normal')(outputs)
  outputs = BatchNormalization()(outputs)
  outputs = ReLU()(outputs)
  outputs = MaxPool2D((2,2))(outputs)

  outputs = Conv2D(256,(3,3),activation=None,padding='same',kernel_initializer='he_normal')(outputs)
  outputs = BatchNormalization()(outputs)
  outputs = ReLU()(outputs)
  outputs = MaxPool2D((2,2))(outputs)

  outputs = Conv2D(512,(3,3),activation=None,padding='same',kernel_initializer='he_normal')(outputs)
  outputs = BatchNormalization()(outputs)
  outputs = ReLU()(outputs)
  outputs = MaxPool2D((2,2))(outputs)

  outputs = GlobalAveragePooling2D()(outputs)
  # outputs = Flatten()(outputs)

  outputs = Dense(128,activation=None,kernel_initializer='he_normal')(outputs)
  outputs = BatchNormalization()(outputs)
  outputs = ReLU()(outputs)
  outputs = Dropout(0.5)(outputs)

  outputs = Dense(32,activation=None,kernel_initializer='he_normal')(outputs)
  outputs = BatchNormalization()(outputs)
  outputs = ReLU()(outputs)
  outputs = Dropout(0.5)(outputs)

  outputs = Dense(11,activation='softmax')(outputs)
  model = Model(inputs=inputs, outputs=outputs)

  # sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
  model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
  
  return model

model = build_model()
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 112, 112, 3)]     0         
                                                                 
 conv2d (Conv2D)             (None, 112, 112, 32)      896       
                                                                 
 batch_normalization (BatchN  (None, 112, 112, 32)     128       
 ormalization)                                                   
                                                                 
 re_lu (ReLU)                (None, 112, 112, 32)      0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 56, 56, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 56, 56, 64)        18496 

In [12]:
filepath= "weights.best_2.hdf5"

callbacks = [
            #  tf.keras.callbacks.EarlyStopping(patience=5, monitor='val_loss'),
             tf.keras.callbacks.TensorBoard(log_dir='./logs', histogram_freq=1),
             tf.keras.callbacks.ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
             ]

             
history = model.fit(agumeted_train, agumeted_train_y,
          batch_size=32,
          epochs=50,
          validation_data=(agumeted_val,agumeted_val_y),
          callbacks=callbacks
          )

Epoch 1/50
215/215 [==============================] - ETA: 0s - loss: 2.4557 - accuracy: 0.1678
Epoch 1: val_loss improved from inf to 3.00788, saving model to weights.best_2.hdf5
215/215 [==============================] - 365s 2s/step - loss: 2.4557 - accuracy: 0.1678 - val_loss: 3.0079 - val_accuracy: 0.1047
Epoch 2/50
215/215 [==============================] - ETA: 0s - loss: 2.0004 - accuracy: 0.2870
Epoch 2: val_loss improved from 3.00788 to 1.82016, saving model to weights.best_2.hdf5
215/215 [==============================] - 384s 2s/step - loss: 2.0004 - accuracy: 0.2870 - val_loss: 1.8202 - val_accuracy: 0.3372
Epoch 3/50
215/215 [==============================] - ETA: 0s - loss: 1.7303 - accuracy: 0.3672
Epoch 3: val_loss did not improve from 1.82016
215/215 [==============================] - 365s 2s/step - loss: 1.7303 - accuracy: 0.3672 - val_loss: 2.2441 - val_accuracy: 0.2093
Epoch 4/50
215/215 [==============================] - ETA: 0s - loss: 1.5069 - accuracy: 0.4434
E

In [16]:
model.evaluate(agumeted_val,agumeted_val_y)

6/6 [==============================] - 2s 337ms/step - loss: 0.6223 - accuracy: 0.9012


[0.6222730875015259, 0.9011628031730652]

In [17]:
test_r = resize_and_rescale(X_test)
pred = model.predict(test_r)
pred

array([[6.18609261e-11, 9.99999762e-01, 8.57324522e-09, ...,
        1.68538804e-14, 4.02544087e-09, 7.40185646e-09],
       [8.95642638e-02, 2.50825584e-02, 5.85428178e-01, ...,
        2.18778114e-05, 1.08862831e-03, 5.33962040e-04],
       [4.87306376e-11, 9.99965310e-01, 4.24248356e-06, ...,
        1.35925071e-12, 2.61476430e-07, 7.38158201e-09],
       ...,
       [9.13113638e-07, 3.43266123e-07, 8.67715385e-03, ...,
        3.69870996e-08, 4.79757079e-09, 1.76335891e-06],
       [2.15812340e-13, 3.08288776e-07, 6.61625421e-10, ...,
        1.14223474e-11, 3.38479005e-08, 4.54064564e-12],
       [8.69453842e-09, 9.98973250e-01, 1.02138973e-03, ...,
        1.83775509e-10, 3.72546545e-08, 4.79002483e-06]], dtype=float32)

In [18]:
sub['label']=np.argmax(pred,axis=1)
sub['label'][sub['label']==10]='10-1'
sub['label'][sub['label']==0]='10-2'
sub['label'].astype('str')

sub.to_csv('/content/drive/MyDrive/user_data/0602.csv',index=False)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
